## Lesson 3

<b>Get Data</b> - Our data set will consist of an Excel file containing customer counts per date. We will learn how to read in the excel file for processing.<br>
<b>Prepare Data</b> - The data is an irregular time series having duplicate dates. We will be challenged in compressing the data and coming up with next years forecasted customer count.<br>
<b>Analyze Data</b> - We use graphs to visualize trends and spot outliers. Some built in computational tools will be used to calculate next years forecasted customer count.<br>
<b>Present Data</b> - The results will be plotted.

<b>NOTE: Make sure you have lookend through all previous lessons, as the knowledge learned in previous lessons will be needed for this exercise.</b>

In [2]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy.random as np
import sys
import matplotlib

%matplotlib inline

In [3]:
print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)
print('Matplotlib version ' + matplotlib.__version__)

Python version 3.7.2 (tags/v3.7.2:9a3ffc0492, Dec 23 2018, 22:20:52) [MSC v.1916 32 bit (Intel)]
Pandas version 0.23.4
Matplotlib version 3.0.2


We will be creating our own test data for analysis.

In [4]:
# set seed
np.seed(111)

# Function to generate test data
def CreateDataSet(Number = 1):
    Output = []
    
    for i in range(Number):
        
        # Create a weekly (mondays) date range
        rng = pd.date_range(start='1/1/2009', end='12/31/2012', freq='W-MON')
        
        # Create random data
        data = np.randint(low=25, high=1000, size=len(rng))
        
        # Status pool
        status = [1, 2, 3]
        
        # Make a random list of statuses
        random_status = [status[np.randint(low=0, high=len(status))] for i in range(len(rng))]
        
        # State pool
        states = ['GA', 'FL', 'fl', 'NY', 'NJ', 'TX']
        
        # Make a random list of states
        random_states = [states[np.randint(low=0, high=len(states))] for i in range(len(rng))]
        
        Output.extend(zip(random_states, random_status, data, rng))
        
    return Output

Now that we have a function to generate our test data, lets create some data and stick it into a dataframe.

In [5]:
dataset = CreateDataSet(4)
df = pd.DataFrame(data = dataset, columns=['State', 'Status', 'CustomerCount', 'StatusDate'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836 entries, 0 to 835
Data columns (total 4 columns):
State            836 non-null object
Status           836 non-null int64
CustomerCount    836 non-null int64
StatusDate       836 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 22.9+ KB


In [6]:
df.head()

,State,Status,CustomerCount,StatusDate
0,GA,1,877,2009-01-05
1,FL,1,901,2009-01-12
2,fl,3,749,2009-01-19
3,FL,3,111,2009-01-26
4,GA,1,300,2009-02-02


We are now going to save this dataframe into an Excel file, to then bring it back to a dataframe. <br>
We simply do this to show you how to read and write to Excel files.<br><br>
We do not write the index values of the dataframe to the Excel file, since they are not meant to be part of our initial test data set.

In [9]:
# Save results to excel : need to openpyxl module. (pip install openpyxl)
df.to_excel('Lesson3.xlsx', index=False)
print('Done')

Done


## Grab Data from Excel

We will be using the <b>read_excel</b> function to read in data from an Excel file. The function allows you to read in specific tabs by name or location.

In [10]:
pd.read_excel?

<b>Note: The location on the Excel file will be in the same folder as the notebook, unless specified otherwise.</b>

In [12]:
# Location of file
Location = r'C:\DevProject\PyProject\jupyter_doc\Lesson3.xlsx'

# Parse a specific sheet
df = pd.read_excel(Location, 0, index_col='StatusDate')
df.dtypes

State            object
Status            int64
CustomerCount     int64
dtype: object

df.index

In [14]:
df.head()

,State,Status,CustomerCount
StatusDate,,,
2009-01-05,GA,1,877
2009-01-12,FL,1,901
2009-01-19,fl,3,749
2009-01-26,FL,3,111
2009-02-02,GA,1,300


## Prepare Data

This section attempts to clean up the data for analysis.<br>
1. Make sure the state column is all in upper case<br>
2. Only select records where the account status is equal to "1"<br>
3. Merge (NJ and NY) to NY in the state column<br>
4. Remove any outliers (any odd results in the data set)

Lets take a quick look on how some of the State values are upper case and some are lower case

In [16]:
df['State'].unique()

array(['GA', 'FL', 'fl', 'TX', 'NY', 'NJ'], dtype=object)